https://neo4j.com/labs/genai-ecosystem/graphrag-python/

https://neo4j.com/docs/neo4j-graphrag-python/current/user_guide_rag.html#

In [1]:
import sys
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler(sys.stdout))

In [36]:
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "anatomy-exodus-pigment-colombo-octopus-1216"

In [37]:
from neo4j_graphrag.generation import RagTemplate

prompt_template = RagTemplate(
    system_instructions="""당신은 전문 지식과 폭넓은 정보를 갖춘 최첨단 에이전트입니다. 사용자가 질문을 하면 다음 지침에 따라 응답하세요.
1. **풍부한 정보 제공:**
   - 질문에 대해 가능한 모든 관련 배경 지식과 맥락을 설명하고, 필요한 경우 구체적인 예시와 참고 사항을 포함하세요.
   - 복잡한 주제는 단계별로 세분화하여 설명하며, 각 단계의 중요 포인트를 명확하게 전달하세요.

2. **체계적이고 명확한 응답:**
   - 응답은 명확한 구조(예: 개요, 세부 사항, 결론)와 논리적 순서를 갖추어 작성하세요.
   - 정보의 출처나 근거가 있을 경우 이를 함께 명시하고, 최신 정보에 기반한 답변을 제공하도록 노력하세요.

3. **사용자 맞춤형 설명:**
   - 질문의 맥락과 사용자의 수준에 맞춰 설명의 깊이와 세부 사항을 조절하세요.
   - 추가 설명이나 예시가 필요한 경우, 부가 설명을 제공하여 이해를 돕도록 하세요.

4. **중립적이고 객관적인 자세:**
   - 모든 정보는 사실에 기반하여 객관적으로 전달하며, 다양한 관점이나 의견이 존재하는 경우 이를 균형 있게 소개하세요.
   - 최신 업데이트나 변화가 있는 경우, 그에 따른 주의사항을 명확히 설명하세요.

5. **대화형 인터랙션 유지:**
   - 사용자의 추가 질문이나 요청에 대해 신속하고 유연하게 대응하며, 필요한 경우 추가적인 정보를 요청하여 보다 정확한 답변을 제공할 수 있도록 하세요.

이러한 지침을 준수하여 사용자가 필요로 하는 풍부하고 심도 있는 정보를 제공하는 것이 목표입니다. 항상 사용자에게 최고의 도움이 될 수 있도록 노력하세요.

Domain url is 'https://www.lguplus.com/'.
    """
)

In [4]:
# Neo4j Driver
import neo4j

neo4j_driver = neo4j.GraphDatabase.driver(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
)

# LLM and Embedding Model
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings

llm = OpenAILLM(
    model_name="gpt-4o",
    model_params={
        "temperature": 0  # turning temperature down for more deterministic results
    }
)

embedder = OpenAIEmbeddings()

In [33]:
# Vector Retriever
from neo4j_graphrag.generation import GraphRAG
from neo4j_graphrag.retrievers import VectorRetriever

vector_retriever = VectorRetriever(
    neo4j_driver,
    index_name="text_embeddings",
    embedder=embedder
)

vector_rag = GraphRAG(llm=llm, retriever=vector_retriever, prompt_template=prompt_template)

result = vector_rag.search("테더링 요금제 추천해줘", retriever_config={'top_k': 5}, return_context=True)
result

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


RagResultModel(answer="온가족이 함께 사용할 수 있는 요금제를 추천하기 위해서는 가족 구성원의 사용 패턴과 필요에 맞는 요금제를 선택하는 것이 중요합니다. 일반적으로 가족 요금제는 데이터 공유, 음성 통화, 문자 메시지 등의 혜택을 포함하여 여러 명이 함께 사용할 수 있도록 설계되어 있습니다. LG U+의 경우, 다양한 가족 요금제를 제공하고 있으며, 그 중 몇 가지를 소개하겠습니다.\n\n1. **가족 데이터 공유 요금제**:\n   - 이 요금제는 가족 구성원들이 데이터를 공유할 수 있는 옵션을 제공합니다. 각 구성원이 개별적으로 데이터를 구매하는 것보다 경제적일 수 있습니다.\n   - 예를 들어, '데이터 함께 쓰기' 옵션을 통해 가족 구성원들이 하나의 데이터 풀을 공유할 수 있습니다.\n\n2. **음성 및 문자 무제한 요금제**:\n   - 가족 구성원들이 음성 통화와 문자 메시지를 무제한으로 사용할 수 있는 요금제입니다. 특히, 자주 통화하는 가족에게 유리합니다.\n\n3. **결합 할인 혜택**:\n   - 여러 명이 함께 가입할 경우, 결합 할인 혜택을 받을 수 있습니다. 이는 가족 구성원들이 각각의 요금제에 가입하는 것보다 비용을 절감할 수 있는 방법입니다.\n\n4. **추가 혜택**:\n   - 가족 요금제에는 종종 OTT 서비스, 음악 스트리밍 서비스 등의 추가 혜택이 포함될 수 있습니다. 예를 들어, '모두의할인팩'과 같은 서비스는 다양한 미디어 상품을 할인된 가격에 제공하여 가족 구성원들이 함께 즐길 수 있습니다.\n\n각 요금제의 구체적인 내용과 조건은 LG U+ 공식 웹사이트(https://www.lguplus.com/)에서 확인할 수 있으며, 고객센터를 통해 상담을 받아 가족의 필요에 가장 적합한 요금제를 선택하는 것이 좋습니다.", retriever_result=RetrieverResult(items=[RetrieverResultItem(content='{\'id\': \'65c346e26a0d167f8832

In [32]:
from neo4j_graphrag.generation import GraphRAG, RagTemplate
from neo4j_graphrag.retrievers import VectorCypherRetriever

graph_retriever = VectorCypherRetriever(
    neo4j_driver,
    index_name="text_embeddings",
    embedder=embedder,
    retrieval_query="""
MATCH (node)-[relList]-{1,2}(nb: Document)
RETURN nb.text
LIMIT 20
"""
)

graph_rag = GraphRAG(llm=llm, retriever=graph_retriever, prompt_template=prompt_template)

result = graph_rag.search("온가족이 함께 쓸 수 있는 요금제 추천해줘", retriever_config={'top_k': 5}, return_context=True)
result

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


RagResultModel(answer='온가족이 함께 사용할 수 있는 요금제를 추천드리겠습니다. LG U+에서는 가족이 함께 데이터를 공유할 수 있는 다양한 5G 요금제를 제공하고 있습니다. 아래는 몇 가지 추천 요금제입니다.\n\n1. **(우리집지킴이 Easy2+) 5G 시그니처**\n   - **기본 요금:** 130,000원\n   - **데이터 제공량:** 무제한\n   - **테더링/쉐어링:** 120GB\n   - **음성 통화:** 집/이동전화 무제한, 부가통화 300분\n   - **혜택:** 스마트홈 우리집지킴이 Easy2+ 서비스 제공\n   - **추가 혜택:** 참 쉬운 가족 데이터 60GB\n   - **설명:** U⁺의 5G 서비스와 스마트홈 서비스를 함께 즐기고, 가족과 공유할 수 있는 데이터를 추가로 받는 5G 가족형 요금제입니다.\n   - **상세 URL:** [요금제 상세보기](https://www.lguplus.com/plan/mplan/5g-all/5g-category/LPZ1003399)\n\n2. **(유튜브 프리미엄) 5G 프리미어 슈퍼**\n   - **기본 요금:** 115,000원\n   - **데이터 제공량:** 무제한\n   - **테더링/쉐어링:** 100GB\n   - **음성 통화:** 집/이동전화 무제한, 부가통화 300분\n   - **혜택:** 유튜브 프리미엄 제공, 미디어 팩 (택1) 무료\n   - **추가 혜택:** 참 쉬운 가족 데이터 50GB\n   - **설명:** U⁺5G 서비스와 유튜브 프리미엄을 할인 받아 마음껏 즐기고 가족 공유 데이터까지 받을 수 있는 요금제입니다.\n   - **상세 URL:** [요금제 상세보기](https://www.lguplus.com/plan/mplan/5g-all/5g-category/LPZ1002317)\n\n이 요금제들은 가족이 함께 데이터를 나누어 사용할 수 있는 기능을 제공하며, 다양한 미디어 서비스 혜택도 포함되어 있어 가족 모두가 만족할 

In [53]:
# Text2Cypher Retriever
from neo4j_graphrag.retrievers import Text2CypherRetriever
from neo4j_graphrag.generation import GraphRAG, RagTemplate

product_schema = {
    "label": "Service",
    "properties": [
        {"name": "id", "type": "string"},
    ]
}

price_schema = {
    "label": "Price",
    "properties": [
        {"name": "id", "type": "string"},
    ]
}

has_price_schema = {
    "type": "HAS_PRICE",
    "source": "Product",
    "target": "Price"
}


neo4j_schema ={
    "nodes": [product_schema, price_schema],
    "relationships": [has_price_schema]
}

text2cypher_retriever = Text2CypherRetriever(
    driver=neo4j_driver,
    llm=llm,
    neo4j_schema=str(neo4j_schema)
)

text2cypher_rag = GraphRAG(
    llm=llm,
    retriever=text2cypher_retriever,
    prompt_template=prompt_template
)

result = text2cypher_rag.search(
    "티빙 베이직 가격 알려줘",
    return_context=True
)
result

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


RagResultModel(answer='티빙 베이직 요금제의 가격은 월 9,500원입니다. 이 요금제는 티빙의 기본적인 콘텐츠를 이용할 수 있는 플랜으로, 다양한 드라마, 영화, 예능 프로그램 등을 시청할 수 있습니다. 티빙은 한국의 대표적인 OTT(Over-The-Top) 서비스 중 하나로, 다양한 장르의 콘텐츠를 제공하여 많은 사용자들에게 인기를 끌고 있습니다. 요금제에 대한 더 자세한 정보나 다른 플랜에 대한 비교가 필요하시면 추가로 문의해 주세요.', retriever_result=RetrieverResult(items=[RetrieverResultItem(content="<Record p=<Node element_id='4:e91334ed-cf36-4fa8-a84c-5afca4e086d3:1012' labels=frozenset({'__Entity__', 'Price'}) properties={'id': '월 9,500원'}>>", metadata=None)], metadata={'cypher': "MATCH (s:Service {id: '티빙 베이직'})-[:HAS_PRICE]->(p:Price) RETURN p", '__retriever': 'Text2CypherRetriever'}))